In [ ]:
from sourced.engine import Engine
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

spark = SparkSession.builder\
.master("local[*]").appName("playground")\
.getOrCreate()

engine = Engine(spark, "repositories")

### Count the total of non-fork repositories

In [ ]:
engine.repositories.filter("is_fork = false")\
.select("id").distinct()\
.count()

### Get all the files of all head commits

In [ ]:
head_files = engine.repositories.filter("is_fork = false").references\
.head_ref.commits.first_reference_commit\
.files\
.classify_languages()\
.filter("is_binary = false")\
.select("file_hash", "path", "content", "lang").filter("lang is not null").cache()

### Get the schema

In [ ]:
head_files.printSchema()

### Print result

In [ ]:
head_files.show()

### Top languages per number of files

In [ ]:
top_ten_langs = head_files.distinct()\
.groupBy("lang").agg(count("*").alias("count"))\
.orderBy("count").sort(desc("count")).limit(10)\
.show()

### Get all Java files

In [ ]:
 head_files.groupBy("lang").agg(count("*").alias("count")).filter("lang='Java'").show()

In [ ]:
head_files.limit(10).show()

### Get identifiers of all Python files

In [ ]:
idents = engine.repositories.filter("is_fork = false").references\
.head_ref.commits.first_reference_commit\
.files\
.classify_languages()\
.extract_uasts()\
.query_uast('//*[@roleIdentifier and not(@roleIncomplete)]')\
.filter("is_binary = false")\
.filter("lang = 'Python'")\
.select("file_hash","result").distinct()

### Get the tokens from the identifiers

In [ ]:
tokens = idents.extract_tokens()

In [ ]:
tokens.limit(10).show()